%pip install lxml
%pip install html5lib

• https://rp5.ru/  прогнозы на 1, 2, 3… 7 суток вперед 

• https://ru-meteo.ru/   прогнозы на 1, 2 суток вперед 

• https://goodmeteo.ru/     прогнозы на 1, 2, 3 суток вперед   

• https://world-weather.ru/pogoda/ прогнозы на 1, 2 суток вперед

In [4]:
from urllib.request import urlopen

from bs4 import BeautifulSoup
from statistics import mean 
import pandas as pd
import re
from datetime import datetime, timedelta
import time
import requests
import os, warnings
from calendar import monthrange

warnings.filterwarnings("ignore")

# yandex.ru

## Посуточный парсер

Пример посуточного парсера
https://github.com/JWscientist/Weather/blob/main/%D0%AF%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%D0%9F%D0%BE%D0%B3%D0%BE%D0%B4%D0%B0%20%D0%BF%D0%B0%D1%80%D1%81%D0%B5%D1%80%20pandas.ipynb

In [11]:
#даты
date_now = datetime.today().date()
day_now = date_now.day #<--- день сегодня
date_tomorrow = date_now + timedelta(days=1)
day_tomorrow = (date_now + timedelta(days=1)).day #<--- день завтра
day_after_tomorrow = (date_now + timedelta(days=2)).day #<--- день послезавтра

In [12]:
#считываем страницу с прогнозом Яндекса через read_html, получаем необработанные данные 
ya_df = pd.read_html('https://yandex.ru/pogoda/details?lat=55.753215&lon=37.622504&via=ms')

In [16]:
#каждый день находится в отдельной ячейке, прилепим их их друг под другом
ya_new=pd.concat([ya_df[0],ya_df[1],ya_df[2],ya_df[3],ya_df[4],ya_df[5],ya_df[6],ya_df[7],ya_df[8],ya_df[9]],ignore_index=True)
ya_new.columns=['temp','nan','weather','press','wet','wind','t_eff']

In [18]:
ya_new

,temp,nan,weather,press,wet,wind,t_eff
0,"утром+7…+17утром, от 7 до 17°",NaN,Облачно с прояснениями,757,43%,"2,1З2, западный",+99°
1,"днём+17…+18днём, от 17 до 18°",NaN,Пасмурно,755,25%,"2,4С2, северный",+1515°
2,"вечером+15…+18вечером, от 15 до 18°",NaN,Пасмурно,754,37%,"1,8С2, северный",+1414°
3,"ночью+9…+14ночью, от 9 до 14°",NaN,Пасмурно,754,57%,"1,4ЮВ1, юго-восточный",+1010°
4,"утром+10…+18утром, от 10 до 18°",NaN,Облачно с прояснениями,755,49%,"3,2С3, северный",+1212°
5,"днём+19днём, 19",NaN,Облачно с прояснениями,754,42%,"3,2СВ3, северо-восточный",+1717°
6,"вечером+15…+19вечером, от 15 до 19°",NaN,Облачно с прояснениями,754,49%,"2,3СЗ2, северо-западный",+1515°
7,"ночью+10…+13ночью, от 10 до 13°",NaN,Облачно с прояснениями,754,67%,"1,5СЗ2, северо-западный",+99°
8,"утром+11…+20утром, от 11 до 20°",NaN,Пасмурно,754,55%,"2,6СЗ3, северо-западный",+1414°
9,"днём+21…+22днём, от 21 до 22°",NaN,Пасмурно,754,33%,"3,4СЗ3, северо-западный",+1919°


In [ ]:
#поработаем со столбцом температуры при помощи регулярных выражений
temp_clean=[]
for i in range(0,ya_new.shape[0]):
    t_str=ya_new['temp'][i]
    temp_str=re.search(r'[+ -]\d{1,2}\…[+ -]\d{1,2}',t_str)
    temp_clean.append(temp_str.group(0))
    
temp_fin=pd.Series(temp_clean) 
temp_fin=temp_fin.str.rsplit('…',expand=True)

temp_fin.columns=['t_down','t_up']   

## Почасовой парсер

In [158]:
class ForecastYandex:
    def __init__(self, forecast_raw):
        
        self.forecast_list = re.findall("(\d{1,2}:\d{2})((\+|\-)\d+)([^\,]*)", forecast_raw)
        
        # try:
        self.__check_time()
        self.data = [{'time': f[0], 
                      'temperature': float(f[1]), 
                      'conditions': re.search('([час]\S*\s)(.*)', f[-1])[2], 
                      'message_debug': f[-1][1:]} for f in self.forecast_list]
        
        # except Exception as e:
        #     print(f'error in __init__: {e}')
    
    @staticmethod
    def __trim_hour(forecast_string):
        # обрезка часа до одной значащей цифры
        wrong_item = list(forecast_string)
        wrong_item[0] = wrong_item[0][-4:]
        
        return tuple(wrong_item)
    
    
    @staticmethod
    def __it_is_correct_delta(delta):
        # проверка, что разница во времени корректна
        allowed_values = (timedelta(days=0, hours=1), 
                          timedelta(days=-1, hours=1))
        return delta in allowed_values


    def __get_timedelta_by_index(self, idx2, idx1):
        # получение разницы во времени (формат '%H:%M') между двумя индексами
        return datetime.strptime(self.forecast_list[idx2][0], '%H:%M') - \
                datetime.strptime(self.forecast_list[idx1][0], '%H:%M')


    def __check_time(self):
        date = datetime.now().date()
        idxs = [datetime.strptime(self.forecast_list[0][0], '%H:%M').replace(date.year, date.month, date.day)]
        
        # Проверка, что все временные метки идут с разницей в 1 час
        
        for i in range(len(self.forecast_list)-1):
            try:
                delta = self.__get_timedelta_by_index(i+1, i)

            except:
                self.forecast_list[i+1] = self.__trim_hour(self.forecast_list[i+1])
                delta = self.__get_timedelta_by_index(i+1, i)
                
                
            if not self.__it_is_correct_delta(delta):
                self.forecast_list[i+1] = self.__trim_hour(self.forecast_list[i+1]) 
                delta = self.__get_timedelta_by_index(i+1, i)
                
                if not self.__it_is_correct_delta(delta):
                    raise AttributeError(f"Wrong time sequence: {self.forecast_list[i][0]}, {self.forecast_list[i+1][0]}")
                    
            if delta == timedelta(days=-1, hours=1):
                date += timedelta(days=1)
            
            idxs.append(datetime.strptime(self.forecast_list[i+1][0], '%H:%M').replace(date.year, date.month, date.day))
            
        self.forecast_list = [(str(dt), *forecast[1:]) for forecast, dt in zip(self.forecast_list, idxs)]            
        
                        
def get_fact_weather(soup):
    result = {}

    for element in soup.find_all('div', {'class': 'card_size_big'}):
        fact_temp = element.find('div', {'class': 'fact__temp-wrap'})
        result['time'] = datetime.now().strftime('%H:%M')

        if fact_temp:
            x = fact_temp.find('a')['aria-label'].split(',')
            result['temperature'] = re.search('(\+|\-)?\d+', x[0])[0]
            result['conditions'] = x[2][:-1].lower()
        else:
            result['temperature'] = None
            result['conditions'] = None

        fact_wind = element.find('div', {'class': 'fact__wind-speed'})
        result['wind_speed'] = fact_wind.find('span', {'class': 'wind-speed'}).text.replace(',', '.') if fact_wind else None
        result['wind_direction'] = fact_wind.find('abbr').text if fact_wind else None

        fact_humidity = element.find('div', {'class': 'fact__humidity'})
        result['humidity'] = re.search('\d{2,3}',fact_humidity.text)[0] if fact_humidity else None


        fact_pressure = element.find('div', {'class': 'fact__pressure'})
        result['pressure'] = re.search('\d{3}',fact_pressure.text)[0] if fact_pressure else None


    return result

### Тестирование

In [47]:
URL = "https://yandex.ru/pogoda/?lat=56.813158&lon=60.643738"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

soup = BeautifulSoup(requests.get(URL, headers=headers).content, 'html5lib')

#### Прогноз

In [76]:
forecast_raw = soup.find_all('ul', {'class': 'swiper-wrapper'})[0].text
forecast_raw

'12:00+28°В 12 часов ясно, +28°13:00+28°В 13 часов облачно с прояснениями, +28°14:00+28°В 14 часов ясно, +28°15:00+29°В 15 часов облачно с прояснениями, +29°16:00+28°В 16 часов облачно с прояснениями, +28°17:00+28°В 17 часов облачно с прояснениями, +28°18:00+28°В 18 часов облачно с прояснениями, +28°19:00+28°В 19 часов облачно с прояснениями, +28°20:00+26°В 20 часов облачно с прояснениями, +26°21:00+23°В 21 час облачно с прояснениями, +23°21:23ЗакатЗакат в 21:2322:00+20°В 22 часа облачно с прояснениями, +20°23:00+17°В 23 часа облачно с прояснениями, +17°чт, 0:00+15°В 0 часов облачно с прояснениями, +15°1:00+14°В 1 час облачно с прояснениями, +14°2:00+13°В 2 часа облачно с прояснениями, +13°3:00+12°В 3 часа ясно, +12°4:00+12°В 4 часа ясно, +12°4:24ВосходВосход в 4:245:00+12°В 5 часов ясно, +12°6:00+14°В 6 часов ясно, +14°7:00+18°В 7 часов облачно с прояснениями, +18°8:00+22°В 8 часов облачно с прояснениями, +22°9:00+24°В 9 часов облачно с прояснениями, +24°10:00+27°В 10 часов облачно с 

In [146]:
A = [('12','434','66'), ('11', '22', '33')]
A

[('12', '434', '66'), ('11', '22', '33')]

In [150]:
[(11, *a[1:]) for a in A]

[(11, '434', '66'), (11, '22', '33')]

In [160]:
forecast.data

[{'time': '2023-05-24 12:00:00',
  'temperature': 28.0,
  'conditions': 'ясно',
  'message_debug': 'В 12 часов ясно'},
 {'time': '2023-05-24 13:00:00',
  'temperature': 28.0,
  'conditions': 'облачно с прояснениями',
  'message_debug': 'В 13 часов облачно с прояснениями'},
 {'time': '2023-05-24 14:00:00',
  'temperature': 28.0,
  'conditions': 'ясно',
  'message_debug': 'В 14 часов ясно'},
 {'time': '2023-05-24 15:00:00',
  'temperature': 29.0,
  'conditions': 'облачно с прояснениями',
  'message_debug': 'В 15 часов облачно с прояснениями'},
 {'time': '2023-05-24 16:00:00',
  'temperature': 28.0,
  'conditions': 'облачно с прояснениями',
  'message_debug': 'В 16 часов облачно с прояснениями'},
 {'time': '2023-05-24 17:00:00',
  'temperature': 28.0,
  'conditions': 'облачно с прояснениями',
  'message_debug': 'В 17 часов облачно с прояснениями'},
 {'time': '2023-05-24 18:00:00',
  'temperature': 28.0,
  'conditions': 'облачно с прояснениями',
  'message_debug': 'В 18 часов облачно с про

In [159]:
forecast = ForecastYandex(forecast_raw)

[('2023-05-24 12:00:00', '+28', '+', '°В 12 часов ясно'), ('2023-05-24 13:00:00', '+28', '+', '°В 13 часов облачно с прояснениями'), ('2023-05-24 14:00:00', '+28', '+', '°В 14 часов ясно'), ('2023-05-24 15:00:00', '+29', '+', '°В 15 часов облачно с прояснениями'), ('2023-05-24 16:00:00', '+28', '+', '°В 16 часов облачно с прояснениями'), ('2023-05-24 17:00:00', '+28', '+', '°В 17 часов облачно с прояснениями'), ('2023-05-24 18:00:00', '+28', '+', '°В 18 часов облачно с прояснениями'), ('2023-05-24 19:00:00', '+28', '+', '°В 19 часов облачно с прояснениями'), ('2023-05-24 20:00:00', '+26', '+', '°В 20 часов облачно с прояснениями'), ('2023-05-24 21:00:00', '+23', '+', '°В 21 час облачно с прояснениями'), ('2023-05-24 22:00:00', '+20', '+', '°В 22 часа облачно с прояснениями'), ('2023-05-24 23:00:00', '+17', '+', '°В 23 часа облачно с прояснениями'), ('2023-05-25 00:00:00', '+15', '+', '°В 0 часов облачно с прояснениями'), ('2023-05-25 01:00:00', '+14', '+', '°В 1 час облачно с прояснени

#### Текущая погода

In [128]:
get_fact_weather(soup)

{'time': '15:59',
 'temperature': '16',
 'conditions': ' облачно с прояснениями',
 'wind_speed': '3.6',
 'wind_direction': 'ЮВ',
 'humidity': '30',
 'pressure': '740'}

### Опрос

In [19]:
URL = "https://yandex.ru/pogoda/?lat=56.813158&lon=60.643738"

soup = BeautifulSoup(requests.get(URL).content, 'html5lib')
forecast_raw = soup.find_all('ul', {'class': 'swiper-wrapper'})[0].text

forecast = ForecastYandex(forecast_raw).data
forecast.insert(0, get_fact_weather(soup))

filename = f'{datetime.now().strftime("%m%d%Y_%H%M")}.csv'
df = pd.DataFrame(forecast)
df.to_csv(filename, index=False)

print(f"{str(datetime.now())} Data successfully saved to {filename}")


2023-05-12 16:11:26.414490 Data successfully saved to 05122023_1611.csv


# world-weather.ru

In [1305]:
URL = "https://world-weather.ru/pogoda/russia/yekaterinburg/24hours/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
#soup = BeautifulSoup(requests.get(URL, headers), 'html5lib')

In [1306]:
requests.get(URL, headers)

<Response [403]>

# goodmeteo.ru

In [271]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

In [272]:
URL = "https://goodmeteo.ru/pogoda-ekaterinburg/"
soup = BeautifulSoup(requests.get(URL).content, 'html5lib')

In [344]:
class ForecastGoodmeteo:
    def __init__(self, forecast_raw):
        
        self.data = pd.DataFrame()

        self.data['temperature'] = forecast_raw['Температура'].apply(lambda x: float(re.search('\-?\d{1,2}(,|.)\d', x)[0].replace(',','.')))
        self.data['wind_direction'] = forecast_raw['Ветер'].apply(lambda x: x.split(', ')[1])
        self.data['wind_speed'] = forecast_raw['Ветер'].apply(lambda x: float(re.search('\d{1,2}(,|.)?\d?', x)[0].replace(',','.')))
        self.data['humidity'] = forecast_raw['Влажность'].apply(lambda x: int(re.search('\d{1,2}', x)[0]))
        self.data['pressure'] = forecast_raw['Давление'].apply(lambda x: int(re.search('\d{3}', x)[0]))
        self.data['cloudiness'] = forecast_raw['Облачность'].apply(lambda x: int(re.search('\d{1,2}', x)[0]))
        self.data['conditions'] = forecast_raw['Осадки']

In [318]:
def get_fact_weather(soup):
    result = {}

    for element in soup.find_all('div', {'class': 'b_pogoda'}):
        result['time'] = str(datetime.now())
        
        det_pog_b1 = element.find('div', {'class': 'det_pog_b1'})
        
        if det_pog_b1:
            det_pog_temp = det_pog_b1.find('div', {'class': 'det_pog_temp'})
            if det_pog_temp:
                result['temperature'] = re.search('(\+|\-)?\d{1,2}', det_pog_temp.text)[0]
                
            det_pog_desc = det_pog_b1.find('div', {'class': 'det_pog_desc'})
            if det_pog_desc:
                result['conditions'] = re.search('([а-яА-Я ])+', det_pog_desc.text)[0].strip()
                
        det_pog_b2 = element.find('div', {'class': 'det_pog_b2'})
        
        if det_pog_b2:
            for element in det_pog_b2.find_all('div'):
                if element.span.text.lower().find('ветер') != -1:
                    text = element.b.text.split(', ')
                    result['wind_speed'] = re.search('\d{1,2}(\,|\.)?\d?', text[0])[0]
                    result['wind_direction'] = text[1]

                    continue

                elif element.span.text.lower().find('влажность') != -1:
                    result['humidity'] = re.search('\d{2}', element.b.text)[0]
                    continue

                elif element.span.text.lower().find('давление') != -1:
                    result['pressure'] = re.search('\d{3}', element.b.text)[0]
                    continue 

    return result

In [321]:
fact_weather = get_fact_weather(soup)
fact_weather

{'time': '2023-05-15 15:13:23.591688',
 'temperature': '+16',
 'conditions': 'слегка облачно',
 'wind_speed': '4',
 'wind_direction': 'Запад',
 'humidity': '42',
 'pressure': '765'}

In [331]:
forecast_filename = 'goodmeteo_forecast.csv'

if not os.path.exists(filename):
    with open(forecast_filename, 'w') as f:
        f.write(','.join(fact_weather.keys())+'\n')
        
with open(forecast_filename, 'a') as f:
    f.write(','.join(fact_weather.values())+'\n')    

In [340]:
URL = "https://goodmeteo.ru/pogoda-ekaterinburg/"
today = pd.read_html(URL, encoding="UTF-8", header=0, index_col=0, parse_dates=True)[0]

URL = "https://goodmeteo.ru/pogoda-ekaterinburg/zavtra/"
tomorrow = pd.read_html(URL, encoding="UTF-8", header=0, index_col=0, parse_dates=True)[0]
tomorrow.index = tomorrow.index + pd.Timedelta('1D')

In [349]:
data = ForecastGoodmeteo(pd.concat([today, tomorrow])).data
data

,temperature,wind_direction,wind_speed,humidity,pressure,cloudiness,conditions
2023-05-15 00:00:00,8.4,Северо-Восток,1.0,90,765,80,пасмурно
2023-05-15 01:00:00,8.7,Юго-Восток,0.8,87,765,83,пасмурно
2023-05-15 02:00:00,9.1,Юг,0.6,84,765,86,пасмурно
2023-05-15 03:00:00,9.7,Север,0.3,79,765,92,сильная облачность
2023-05-15 04:00:00,10.7,Северо-Запад,0.6,75,765,90,сильная облачность
2023-05-15 05:00:00,11.7,Северо-Запад,0.8,71,765,89,сильная облачность
2023-05-15 06:00:00,13.6,Северо-Запад,1.4,63,765,87,сильная облачность
2023-05-15 07:00:00,14.2,Северо-Запад,1.8,58,765,73,сильная облачность
2023-05-15 08:00:00,14.7,Северо-Запад,2.2,54,765,60,сильная облачность
2023-05-15 09:00:00,15.8,Северо-Запад,3.0,45,765,34,облачно


# ru-meteo.ru

In [1307]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
URL = "https://ru-meteo.ru/ekaterinburg/hour"
soup = BeautifulSoup(requests.get(URL).content, 'html5lib')

In [1318]:
class ForecastRumeteo:
    def __init__(self, forecast_raw):
        
        self.data = pd.DataFrame()        
        
        self.data['temperature'] = forecast_raw['column0'].apply(lambda x: re.search('(\+|\-)\d+\.?\d?', x)[0])
        self.data['conditions'] = forecast_raw['column1'].apply(lambda x: x.replace(',', ''))
        self.data['precipitation'] = forecast_raw['Осадки']
        self.data['wind_speed'] = forecast_raw['Ветер'].apply(lambda x: int(re.search('\d{1,2}', x)[0]))
        self.data['wind_direction'] = forecast_raw['Ветер'].apply(lambda x: x.split(', ')[1])
        self.data['pressure'] = forecast_raw['Давление']
        self.data['humidity'] = forecast_raw['Влажность'].apply(lambda x: int(re.search('\d{2,3}', x)[0]))
        
def forecast_from_table(forecast, add_day=-1):
    out = forecast.drop(forecast.shape[0]-1).dropna(how='all', axis=1)
    out.index = out[out.columns[0]].apply(lambda x: pd.to_datetime(re.search('\d{2}\:\d{2}', x)[0]) + pd.to_timedelta(f'{add_day+1}D'))
    out.index.name = 'time'
    out.columns = ['column0', 'column1', 'Осадки', 'Ветер', 'Давление', 'Влажность']
    
    return out

def get_fact_weather(soup):
    result = {}

    for element in soup.find_all('div', {'class': 'content'}):
        result['time'] = str(datetime.now())
        
        last_report = element.find('div', {'class': 'wrap_content'}).find('div', {'class': 'last-report'})
        
        if last_report:
            temp = last_report.find('div', {'class': 'current-temp'})      
            result['temperature'] = re.search('(\+|\-)?\d{1,2}', temp.text)[0] if temp else None
                
        conditions = element.find('div', {'class': 'wrap_content'}).find('div', {'class': 'conditions'})
        
        if conditions:
            result['conditions'] = conditions.find('li', {'class': 'condition-descr'}).text
            
            wind = re.search('(\d)\s\м\/\с\,\s([а-я]+(?=давление:))', conditions.text.lower())
            print(wind)
            try:
                result['wind_speed'] = wind[1]
                result['wind_direction'] = wind[2]
                
            except Exception as e:
                print(f'String: {conditions.text}')
                print(e)
                result['wind_speed'] = None
                result['wind_direction'] = None
                
            try:
                result['pressure'] = re.search('давление:\s(\d{3})', conditions.text.lower())[1]
                
            except Exception as e:
                print(f'String: {conditions.text}')
                print(e)
                result['pressure'] = None
                
            try:
                result['humidity'] = re.search('влажность\s[а-я]+\:\s*(\d{2,3})', conditions.text.lower())[1]
                
            except Exception as e:
                print(f'String: {conditions.text}')
                print(e)
                result['humidity'] = None
                            
        
        ext = element.find('div', {'class': 'wrap_content'}).find('div', {'class': 'ext'})
        if ext:
            try:
                result['visibility'] = re.search('видимость:\s*(([а-я]+\s)+\d+\s[а-я]{1,2})', ext.text)[1]
                
            except Exception as e:
                print(f'String: {ext.text}')
                print(e)
                result['visibility'] = None
        


    return result


In [1338]:
fact_weather = get_fact_weather(soup)
fact_weather

None
String: ясноВетер: 6 м/с, восточный, юго-восточныйДавление: 744 мм рт.ст.Влажность воздуха: 26%Точка росы: −2°CГоризонтал. видимость: более 10 кмГеомагнитный фон спокойныйВосход был: 11 часов назад в 04:33Закат будет: через 5 часов в 21:15Сводка зафиксирована 23 мин назад (в 15:30), в аэропорту Кольцово, расположенном в 16 км от центра города
'NoneType' object is not subscriptable


{'time': '2023-05-19 16:01:00.946360',
 'temperature': '+18',
 'conditions': 'ясно',
 'wind_speed': None,
 'wind_direction': None,
 'pressure': '744',
 'humidity': '26',
 'visibility': 'более 10 км'}

In [1317]:
URL = "https://ru-meteo.ru/ekaterinburg/hour"
forecast = pd.read_html(URL, encoding="UTF-8", header=0)

In [659]:
forecast_raw = forecast_from_table(forecast[0])

for i, df in enumerate(forecast[1:]):
    next_forecast = forecast_from_table(df, i)
    forecast_raw = pd.concat([forecast_raw, next_forecast])  
    
data = ForecastRumeteo(forecast_raw).data
data

,temperature,conditions,precipitation,wind_speed,wind_direction,pressure,humidity
time,,,,,,,
2023-05-16 13:00:00,+21.5,Облачно,0,4,С,740,22
2023-05-16 14:00:00,+21.1,Переменная облачность,0,4,С,740,23
2023-05-16 15:00:00,+21.6,Облачно,0,4,С,740,23
2023-05-16 16:00:00,+21.2,Облачно,0,5,С,740,24
2023-05-16 17:00:00,+20.8,Облачно,0,4,С,740,25
2023-05-16 18:00:00,+20.5,Облачно,0,4,С,740,26
2023-05-16 19:00:00,+19.8,Облачно,0,4,С,740,27
2023-05-16 20:00:00,+18.5,Облачно,0,4,СВ,741,30
2023-05-16 21:00:00,+16.6,Переменная облачность,0,3,СВ,741,34


# rp5.ru

In [1407]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
URL = "https://rp5.ru/%D0%9F%D0%BE%D0%B3%D0%BE%D0%B4%D0%B0_%D0%B2_%D0%95%D0%BA%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%BD%D0%B1%D1%83%D1%80%D0%B3%D0%B5"
soup = BeautifulSoup(requests.get(URL).content, 'html5lib')

In [1444]:
class ForecastRp5:
    
    def __init__(self, forecast_raw, soup, forecast_table = 'forecastTable_1_3'): 
        
        self._forecast_table = forecast_table
        self.soup = soup
        self.data = pd.DataFrame()
        
        self.data['temperature'] = forecast_raw['Температура, °C']
        self.data['wind_speed'] = forecast_raw['Ветер: скорость, м/с']
        self.data['wind_direction'] = forecast_raw['направление']
        self.data['pressure'] = forecast_raw['Давление, мм рт. ст.']
        
        self.data['conditions'], \
        self.data['cloudiness'] = self.__get_cloudiness()
        self.data['precipitation'], \
        self.data['precipitation_info'] = self.__get_precipitation()
        self.data['humidity'] = self.__get_humidity()
        
        
        self.data.index = self.__get_datetime_indexes(forecast_raw.index, forecast_raw['Местное время'])
        self.data.index.name = 'time'     
        
      
        
    def __get_datetime_indexes(self, day_index, hour):
        days = [re.search('\d{1,2}\s\w+', x)[0] for x in day_index]

        day_now = datetime.now().day
        month_now = datetime.now().month
        year_now = datetime.now().year


        days_before_end = monthrange(year_now, month_now)[1] - day_now
        days_diff = [int(day.split()[0]) - day_now if int(day.split()[0]) >= day_now else int(day.split()[0]) + days_before_end for day in days]

        index = [datetime.strptime(f"{year_now}-{month_now}-{day_now} {h}:00", '%Y-%m-%d %H:%M') + timedelta(days=d) for d, h in zip(days_diff, hour)]

        return index
    
    def __get_cloudiness(self, raw_number=2):
        # Облачность
        table = soup.find('table', {'id': self._forecast_table}).find_all('tr')[raw_number].find_all('td')[1:-1]
        result = {'brief': [], 'detailed': []}
        
        for i, element in enumerate(table):
            try:
                result_raw = element.find('div', {'class': 'cc_0'}).find_all('div')[0]['onmouseover']  
                result_info = re.search("<b>(.+)</b><br/>\(([^')]+)", result_raw)
                result['brief'].append(result_info[1] if result_raw else None)
                result['detailed'].append(result_info[2] if result_raw else None)         

            except Exception as e:
                print(f'Error while parsing cloudiness in table column {i}:', e)
                result['brief'].append(None)
                result['detailed'].append(None)
                continue
                
        return result['brief'], result['detailed']
    
    def __get_precipitation(self, raw_number=3):
        table = soup.find('table', {'id': self._forecast_table}).find_all('tr')[raw_number].find_all('td')[1:-1] # Осадки
        result = {'brief': [], 'detailed': []}
        
        for i, element in enumerate(table):
            try:
                result_raw = table_precipitation[i].find('div', {'class': 'pr_0'})['onmouseover']
                result_info = re.search("((?:\w+\s?)+)(\(.+\))?", result_raw.split(", '")[1])
                result['brief'].append(result_info[1] if result_raw else None)
                result['detailed'].append(result_info[2] if result_info[2] else None)       

            except Exception as e:
                print(f'Error while parsing precipitation in table column {i}:', e)
                result['brief'].append(None)
                result['detailed'].append(None)
                continue
                
        return result['brief'], result['detailed']
    
    def __find_forecast_row_by_name(self, name):
        for row in soup.find('table', {'id': 'forecastTable_1_3'}).find_all('tr'):
            if row.find_all('td')[0].text.lower().find(name) != -1:
                return row
        
    def __get_humidity(self, raw_number=10):
        
        table = self.__find_forecast_row_by_name('влажность').find_all('td')[1:-1] # Влажность
        result = []
        
        for i, element in enumerate(table):
            try:
                result.append(table[i].text)       

            except Exception as e:
                print(f'Error while parsing humidity in table column {i}:', e)
                humidity.append(None)
                continue
                
        return result

In [1440]:
forecast_raw = pd.read_html(URL, encoding="UTF-8", header = 0, attrs = {'id': forecast_table})[0]
forecast_raw.index = forecast_raw.iloc[:, 0]
forecast_raw = forecast_raw.iloc[:, 1:-1].T
forecast_raw

"Сегодня, понедельник, 22 мая",Местное время,"Облачность, %","Осадки, мм","Температура, °C","Давление, мм рт. ст.","Ветер: скорость, м/с","порывы, м/с",направление
"Сегодня, понедельник, 22 мая.1",16,NaN,NaN,+25,746,1,6,Ю-В
"Сегодня, понедельник, 22 мая.2",17,NaN,NaN,+25,746,2,7,Ю-В
"Сегодня, понедельник, 22 мая.3",18,NaN,NaN,+25,746,2,7,Ю-В
"Сегодня, понедельник, 22 мая.4",19,NaN,NaN,+23,746,3,7,Ю-В
"Сегодня, понедельник, 22 мая.5",20,NaN,NaN,+23,746,3,6,Ю-В
...,...,...,...,...,...,...,...,...
"Воскресенье, 28 мая.1",05,NaN,NaN,+17,739,2,NaN,Ю
"Воскресенье, 28 мая.2",11,NaN,NaN,+28,740,3,6,Ю-В
"Воскресенье, 28 мая.3",17,NaN,NaN,+30,740,4,9,Ю
"Воскресенье, 28 мая.4",23,NaN,NaN,+24,740,2,5,Ю


In [1446]:
forecast = ForecastRp5(forecast_raw, soup)
forecast.data

,temperature,wind_speed,wind_direction,pressure,conditions,cloudiness,precipitation,precipitation_info,humidity
time,,,,,,,,,
2023-05-22 16:00:00,+25,1,Ю-В,746,Переменная облачность,"облака среднего яруса 32%, верхнего яруса, про...",Явления погоды отсутствуют,None,12
2023-05-22 17:00:00,+25,2,Ю-В,746,Облачно с прояснениями,облака среднего яруса 61%,Явления погоды отсутствуют,None,13
2023-05-22 18:00:00,+25,2,Ю-В,746,Облачно,"облака среднего яруса 80%, верхнего яруса, про...",Явления погоды отсутствуют,None,15
2023-05-22 19:00:00,+23,3,Ю-В,746,Облачно с прояснениями,"облака среднего яруса 67%, верхнего яруса, про...",Явления погоды отсутствуют,None,16
2023-05-22 20:00:00,+23,3,Ю-В,746,Облачно с прояснениями,"облака среднего яруса 70%, верхнего яруса, про...",Явления погоды отсутствуют,None,19
...,...,...,...,...,...,...,...,...,...
2023-05-28 05:00:00,+17,2,Ю,739,Облачно,"облака среднего яруса 79%, верхнего яруса, про...",Явления погоды отсутствуют,None,39
2023-05-28 11:00:00,+28,3,Ю-В,740,Облачно с прояснениями,"облака среднего яруса 67%, верхнего яруса, про...",Явления погоды отсутствуют,None,29
2023-05-28 17:00:00,+30,4,Ю,740,Переменная облачность,облака среднего яруса 40%,Явления погоды отсутствуют,None,10


In [1360]:
class A:
    def __init__(self):
        print(type(self).__name__)
        
class B_A(A):
    def somemethod(self):
        print('method')
        
a = A()

A
